In [1]:
from stance_processor.extract_data_tables import *
from stance_processor.data_extractor import *
from stance_processor.data_loader import *
from stance_processor.cotraining_steps import *
from stance_processor.ConversationsTextSVMClassifier import *
# from stance_processor.ConversationsTextNeuralNetworkClassifier import *

event_seed_tag_labels= { 'Iran_Deal':('../stance_dataset/contentious_data/', 
      { 'thankyoutrump':-1 , 'iranuprising': 1, 'freeiran': 1}, 
      'labeled_users/Iran_Deal_labels_combined.txt' ),

      'Student_Marches':('../stance_dataset/contentious_data/', 
      { 'defendthesecond' : 1, '2ashallnotbeinfringed': 1,'guncontrolnow':-1, 'gunreformnow': -1,
                          'guncontrolnow': -1,  'marchforourlives': -1,'2adefenders': 1, '2ndamendment': 1}, 
      'labeled_users/Student_Marches_labels_combined.txt' ),

      'Santa_Fe_Shooting':('../stance_dataset/contentious_data/', 
      { 'defendthesecond' : 1, '2ashallnotbeinfringed': 1,'guncontrolnow':-1, 'gunreformnow': -1,
                          'guncontrolnow': -1,  'marchforourlives': -1,'2adefenders': 1, '2ndamendment': 1})
    }
        
        
event_name = 'Student_Marches'
# tweets_path = '../stance_dataset/rawdata/fauci_raw/'  # not needed if data extraction is not required
results_path = './results/'
extracted_files_path = '/Users/sumeet_kumar/Library/CloudStorage/OneDrive-IndianSchoolofBusiness/Research_projects/Stance/Stance/stance_dataset/contentious_data/'

seed_tag_labels = { 'defendthesecond' : 1, '2ashallnotbeinfringed': 1,'guncontrolnow':-1, 'gunreformnow': -1,
                   'guncontrolnow': -1,  'marchforourlives': -1,'2adefenders': 1, '2ndamendment': 1}

label_file_name = '/Users/sumeet_kumar/Library/CloudStorage/OneDrive-IndianSchoolofBusiness/Research_projects/Stance/Stance/stance_dataset/labeled_users/' + event_name  + '_labels_combined.txt' #If not, leave blank


seed_tag_labels


{'defendthesecond': 1,
 '2ashallnotbeinfringed': 1,
 'guncontrolnow': -1,
 'gunreformnow': -1,
 'marchforourlives': -1,
 '2adefenders': 1,
 '2ndamendment': 1}

In [2]:
# Read twitter data and create new files (moslty csv and json files)
# dc = DataTablesCreator(tweets_path, event_name, extracted_files_path)
# dc.extract_data()

# Read the newly created files and load them to memory
event_predicted_user_stance = {}
event_predicted_user_stance_conf = {}

for event_name in ['Santa_Fe_Shooting', 'Student_Marches' , 'Iran_Deal']:
    
    seed_tag_labels = event_seed_tag_labels[event_name][1]
    print(seed_tag_labels)
    
    label_file_name = '/Users/sumeet_kumar/Library/CloudStorage/OneDrive-IndianSchoolofBusiness/Research_projects/Stance/Stance/stance_dataset/labeled_users/' + event_name  + '_labels_combined.txt' #If not, leave blank    
    
    event_data = DataTablesExtractor.get_event_data(all_events = [event_name], # 'open_us',
                                 output_path = results_path, #timeline
                                 data_folder = extracted_files_path ,
                                label_folder_path = '/Users/sumeet_kumar/Library/CloudStorage/OneDrive-IndianSchoolofBusiness/Research_projects/Stance/Stance/stance_dataset/labeled_users/',
                               label_file_extension =  '_labels_combined.txt' ,
                               use_labeled_users_only = False)    #




    # Set the parameters for the model
    print(seed_tag_labels, label_file_name, event_name )
    dataJson = event_data[event_name] # loadBiasWatchJson(folder_path, text_file_name, label_file_name )
    
    
    args = {}
    args['binarize'] = False
    args['retweet_count'] = 5000
    args['hashtag_count'] = 250#250
    args['inner_iteration_count']  = 5
    args['iteration_count'] = 3
    args['property_type'] = 2

    args['text_threshold'] = 0.85 # 0.65         
    args['property_confidence_threshold'] = 0.8
    args['confidence_threshold_for_mixing'] = 0.7
    mixing_percent = args['mixing_percent'] = 0.05

    args['seed_tag_labels'] = seed_tag_labels        
    args['name'], args['clf'], args['parameters'] =  ('svm12', Pipeline([('vect', CountVectorizer()),
                                         ('tfidf', TfidfTransformer()),
                                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                                                   alpha=1e-3, max_iter=15, random_state=42)),
                                        ])
                                    , {'vect__ngram_range': [(1, 1), (1, 2)], 
                                      'tfidf__use_idf': (True, False),
                                      'clf-svm__alpha': (1e-2, 1e-3),
                                     })

    args['experiment_type'] = event_name
    args['dataJson'] = dataJson        

    args['results_path'] = results_path


    # Run the model that creates output files
    predicted_user_stance, predicted_user_stance_conf = run_cotrain_experiments(args)


    event_predicted_user_stance[event_name] = predicted_user_stance
    
    event_predicted_user_stance_conf[event_name] = predicted_user_stance_conf    
    
    

{'defendthesecond': 1, '2ashallnotbeinfringed': 1, 'guncontrolnow': -1, 'gunreformnow': -1, 'marchforourlives': -1, '2adefenders': 1, '2ndamendment': 1}
error endtag count
error user count
error endtag count
error source_tweet_user count
error retweet_user count
error source_user count
error source_user count
error reply_user count
user_tweet_count:  151235
user_retweet_count:   2781
user_endtag_count:  149364
endtag_count:  45057
user_endtag_count examples:  ['user', '4852311119', '1942377590', '18022791', '887588053', '390140934', '780626889916321792', '141457038', '872806967093473282', '960196633089646592', '41628696', '4034705230', '385688415', '871149485715365888', '457521902', '2392801119', '1060074588', '887442796268978176', '959797135158935553', '45218857']
user_retweet_count exampless:  ['source_tweet_user', '53692193', '1317129865', '855007514621911040', '878087280321335298', '25073877', '727264708776980480', '3493563087', '292929271', '810619093749559296', '17564591', '26642

In [ ]:
## Can we update user embeddings based on stance in text conversations
## Users having opposing stance, pull them apart.
## Users havgng supposrting stance, pull them together


## Also get stance predictions based on text


In [9]:
df = pd.read_csv('./data/Contentious_pairs_all.csv', encoding="utf-8", 
                 dtype={"response_id":str, "target_id":str,
                       "response_user":str, "target_user":str} ).fillna('')


# df = pd.read_csv("./data/contentious_pairs_with_gpt_ada_embeddings_and_user_embeddings_v2.csv")

# df
df

,event,response_id,target_id,interaction_type,label,label_expanded,Confidence_Level,response_text,target_text,truncated,response_text_ada_embedding,target_text_ada_embedding,target_user,response_user,target_user_embedding,response_user_embedding,target_user_embedding_8,response_user_embedding_8
0,Santa_Fe_Shooting,997626508050157568,997598447376175104,Quote,Support,Implicit_Support,1.0,"Seriously, wtf is wrong with our political sys...",More children have been killed in schools this...,False,"[-0.025092221796512604, -0.013276644051074982,...","[-0.008838048204779625, -0.003347944701090455,...",321631653,18655355,"[0.16930730640888214, -0.4844645857810974, 0.2...","[0.1620851755142212, -0.4702264964580536, 0.21...","[0.16930730640888214, -0.4844645857810974, 0.2...","[0.1620851755142212, -0.4702264964580536, 0.21..."
1,Santa_Fe_Shooting,997575042027458561,997573240380968961,Quote,Denial,Explicit_Denial,1.0,Ma calls BS! https://t.co/bodEWN5Q4C,Former GOP Rep. Jason Chaffetz: 'Politically c...,False,"[-0.033440690487623215, 0.007639116141945124, ...","[-0.027197668328881264, 0.0034390492364764214,...",2467720274,3176702526,"[0.16930730640888214, -0.4844645857810974, 0.2...","[0.2439490556716919, -0.44223514199256897, 0.1...","[0.16930730640888214, -0.4844645857810974, 0.2...","[0.2439490556716919, -0.44223514199256897, 0.1..."
2,Santa_Fe_Shooting,997540582846271494,997535659870117888,Quote,Denial,Explicit_Denial,1.0,"On average, there’s one fake stat about school...","On average, that’s one school shooting every w...",False,"[-0.023548724129796028, 0.022269316017627716, ...","[-0.0044773295521736145, -0.014855715446174145...",24939978,1646856415,"[0.16930730640888214, -0.4844645857810974, 0.2...","[0.1620851755142212, -0.4702264964580536, 0.21...","[0.16930730640888214, -0.4844645857810974, 0.2...","[0.1620851755142212, -0.4702264964580536, 0.21..."
3,General_Terms,1018569947817992192,1017909301040635904,Quote,Denial,Implicit_Denial,1.0,ONE MIGHT BE MADE UP AND NOT REAL. NOT SURE. S...,I’m so confused... - When we were attacked on ...,False,"[0.004989785607904196, -0.004917326848953962, ...","[-0.013004736974835396, -0.01780531369149685, ...",2932942216,766475610059317248,"[0.16930730640888214, -0.4844645857810974, 0.2...","[0.1620851755142212, -0.4702264964580536, 0.21...","[0.16930730640888214, -0.4844645857810974, 0.2...","[0.1620851755142212, -0.4702264964580536, 0.21..."
4,General_Terms,1019395289575239680,1017919759474622464,Quote,Denial,Explicit_Denial,1.0,"False, adult friendships are M-F, 9-5 https://...",Adult friendships https://t.co/Cn3r9l4pZJ,False,"[0.023379407823085785, 0.010180916637182236, -...","[0.01642121560871601, 0.0012021968141198158, 0...",2909998207,59621769,"[0.16930730640888214, -0.4844645857810974, 0.2...","[0.1620851755142212, -0.4702264964580536, 0.21...","[0.16930730640888214, -0.4844645857810974, 0.2...","[0.1620851755142212, -0.4702264964580536, 0.21..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5045,Santa_Fe_Shooting,997648170774654976,997645387669299200,Reply,Comment,Comment,2.0,@cenkuygur @realDonaldTrump @NRA NRA is funded...,"Hey #MAGA guys, I thought @realDonaldTrump was...",False,"[-0.03429972380399704, 0.0034139070194214582, ...","[-0.039812665432691574, 0.005698402877897024, ...",429227921,2948973487,"[0.16930730640888214, -0.4844645857810974, 0.2...","[0.1620851755142212, -0.4702264964580536, 0.21...","[0.16930730640888214, -0.4844645857810974, 0.2...","[0.1620851755142212, -0.4702264964580536, 0.21..."
5046,Santa_Fe_Shooting,998085810636259328,997484633695469570,Reply,Comment,Comment,2.0,@ScottMcGrew There is no right answer to this ...,Guns and prayers: A man shows up to the Santa ...,False,"[-0.023522792384028435, 0.003550187451764941, ...","[-0.017941860482096672, 0.003447071649134159, ...",5438322,103706087,"[0.16930730640888214, -0.4844645857810974, 0.2...","[0.1620851755142212, -0.4702264964580536, 0.21...","[0.16930730640888214, -0.48446458578109

In [21]:



for event_name in ['Student_Marches', 'Santa_Fe_Shooting',  'Iran_Deal']:
    node_present = 0
    all_node_count = 0
    
    target_nodes = []
    target_stances = []
    
    df_filtered = df[df['event'] == event_name]
    
    user_stance = event_predicted_user_stance[event_name]
    
    for target_node in df_filtered['target_user']:
        all_node_count += 1        
        target_nodes.append(str(target_node))
        if str(target_node) in user_stance:
            node_present += 1
            target_stances.append(user_stance[str(target_node)])
        else:
            target_stances.append(0)
            
    response_nodes = []
    response_stances = []
            
    for response_node in df_filtered['response_user']:
        all_node_count += 1      
        response_nodes.append(str(response_node))   
        
        if str(response_node) in user_stance:
            node_present += 1
            response_stances.append(user_stance[str(response_node)])
        else:
            response_stances.append(0)
            
    print(event_name, node_present, all_node_count)
    
    stance_df = pd.DataFrame({'target_user': target_nodes, 'target_user_stance':target_stances ,
                             'response_user': response_nodes, 'response_user_stance': response_stances})
    
    
    stance_df.to_csv('./data/' + event_name + '_userstances.csv')
    

    
    
        

Student_Marches 1189 1434
Santa_Fe_Shooting 2138 2394
Iran_Deal 2174 2408
